In [1]:
%reload_ext autoreload
%autoreload 2


In [2]:
import sys
sys.path.insert(0, r'/Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/build')

import numpy as np

import cpp_probabilities

## Old get_counts

In [3]:
from result_saver import SaverProvider
provider = SaverProvider()
backend = provider.get_backend('ibmq_mumbai') 
job = provider.retrieve_job("cmyhbrqrmwhg008bs4h0") # Mumbai job
# job = provider.retrieve_job("cn6g47862r90008810pg") # Sherbrooke job
memory = job.result().get_memory()

print(memory.shape)


(1000, 9)


In [4]:
from soft_info import get_counts

counts_default = get_counts(memory)
print(counts_default)

10:00:39 Warning: Missing layout or synd_rounds, estimating outcomes without KDEs.. IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/src/soft_info/Probabilities/probabilities.py, LINE: 67
10:00:39 Warning: Not enough kernels or no scaler provided. Using the magnitude of the real part for estimation... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/src/soft_info/Probabilities/probabilities.py, LINE: 28
{'000000000': 622, '000100000': 95, '000000010': 65, '000001000': 61, '000101000': 20, '000101010': 15, '000100010': 12, '000001010': 9, '001000000': 9, '000010101': 7, '010000000': 6, '010110011': 5, '100101000': 5, '100100010': 5, '000010000': 5, '000000100': 5, '100001000': 5, '001000100': 4, '001010001': 3, '001010000': 3, '100100000': 3, '010100100': 2, '000110101': 2, '000000110': 2, '100101010': 2, '100000000': 2, '100001010': 2, '001011000': 2, '000011000': 2, '000010100': 2, '010110010': 1, '0100110

In [8]:
from soft_info import get_KDEs, get_repcode_layout, get_repcode_IQ_map

layout = [25, 19, 26, 22, 16] # To implement: into scratch job_data
# layout = get_repcode_layout(30, provider.get_backend('ibm_sherbrooke') )
# kde_dict, scaler_dict = get_KDEs(provider, 'ibm_sherbrooke', list(range(127)), bandwidths=0.2, plot=False)
kde_dict, scaler_dict = get_KDEs(provider, 'ibmq_mumbai', list(range(27)), bandwidths=0.2, plot=False)

# counts_kde = get_counts(memory, kde_dict, scaler_dict, layout, 3, verbose=True)
#counts_kde = get_counts(memory, kde_dict, scaler_dict, layout, 35, verbose=True) # hardcoded num of rounds can be retrieved from metadata
# (counts_kde)


# synd_rounds = 35
synd_rounds = 3
print("generating the qubit mapping")
qubit_mapping = get_repcode_IQ_map(layout, synd_rounds) #Hardcoded for repetition codes

generating the qubit mapping


# Rescale cpp

In [9]:
# GENERATE GRID DATA

import numpy as np

grid_dict = {}
num_points = 10
for qubit_idx, (kde_0, kde_1) in kde_dict.items():
    scaler = scaler_dict[qubit_idx]

    # Retrieve the dataset for this qubit and split into real and imaginary parts
    data = np.array(memory[0])  # TODO change that: Retrieving data from memory 0 to get the min and max values
    data_real_imag = np.column_stack([np.real(data), np.imag(data)])

    # Scale data
    scaled_data = scaler.transform(data_real_imag)

    # Create grid
    grid_x, grid_y = np.linspace(scaled_data[:, 0].min() - 1, scaled_data[:, 0].max() + 1, num_points), \
                     np.linspace(scaled_data[:, 1].min() - 1, scaled_data[:, 1].max() + 1, num_points)
    grid_x, grid_y = np.meshgrid(grid_x, grid_y)
    grid_points = np.vstack([grid_x.ravel(), grid_y.ravel()]).T

    # Evaluate KDE on the grid for both states
    grid_density_0 = (kde_0.score_samples(grid_points)).reshape(grid_x.shape)
    grid_density_1 = (kde_1.score_samples(grid_points)).reshape(grid_x.shape)
    
    # Create an instance of GridData and store in dictionary
    grid_dict[qubit_idx] = cpp_probabilities.GridData(grid_x, grid_y, grid_density_0, grid_density_1)

In [10]:
def process_scaler_dict(scaler_dict):
    processed_dict = {}
    for qubit_idx, scaler in scaler_dict.items():
        # Assuming the scaler is fit on complex data with real and imaginary parts as separate features
        # Hence, the mean_ and scale_ arrays should have two elements each
        if len(scaler.mean_) != 2 or len(scaler.scale_) != 2:
            raise ValueError(f"Scaler for qubit {qubit_idx} is not fit on complex data.")
        
        mean_real, mean_imag = scaler.mean_
        std_real, std_imag = scaler.scale_
        processed_dict[qubit_idx] = ((mean_real, std_real), (mean_imag, std_imag))

    return processed_dict

processed_scaler_dict = process_scaler_dict(scaler_dict)

print(processed_scaler_dict)

{0: ((16191256.63047, 79778555.89041053), (-290404428.44672, 28557074.80284494)), 1: ((5646029.46578, 50568255.50599121), (-186472147.246225, 16267425.019498207)), 2: ((-4238816.391285, 88160499.3805419), (-391237681.971145, 29032984.73295442)), 3: ((-12240638.5723, 155118328.32731992), (-601195720.27544, 26260367.881752755)), 4: ((-22495357.2465, 76163548.3039887), (-283866370.470445, 25748324.91243394)), 5: ((-14245404.47146, 55197760.24809961), (-196167856.96216, 19290952.08405311)), 6: ((-36802784.474975, 60721563.428868), (-214231587.28699, 17274180.5970455)), 7: ((-21676680.22909, 92781647.52589713), (-454609690.78202, 31299065.422852404)), 8: ((-32365717.730365, 74614953.78947219), (-366640585.3899, 21497304.853535526)), 9: ((-10388631.422875, 58098621.79971227), (-225683537.41576, 18409504.110770237)), 10: ((1189200.634215, 27779842.93438666), (-99880449.50942, 10234355.218898227)), 11: ((-8313140.09236, 22722333.94450446), (-98752547.272965, 9961866.585303377)), 12: ((-5247261

In [12]:
counts = cpp_probabilities.get_counts(memory, qubit_mapping, grid_dict, processed_scaler_dict, synd_rounds)
(sorted(counts.items(), key=lambda x: x[1], reverse=True))

[('000 00 00 00', 857),
 ('000 01 00 00', 18),
 ('000 00 01 00', 11),
 ('000 01 01 01', 11),
 ('000 10 10 10', 11),
 ('001 00 00 00', 8),
 ('100 00 10 00', 8),
 ('100 10 00 10', 7),
 ('000 10 00 00', 6),
 ('000 10 10 00', 6),
 ('001 01 00 01', 5),
 ('010 11 00 11', 5),
 ('000 00 00 01', 4),
 ('001 00 01 00', 4),
 ('001 01 00 00', 4),
 ('100 10 00 00', 4),
 ('000 00 10 00', 3),
 ('010 00 00 00', 3),
 ('100 00 00 00', 3),
 ('000 00 10 10', 2),
 ('000 01 01 00', 2),
 ('000 10 00 10', 2),
 ('000 00 00 10', 1),
 ('000 11 01 01', 1),
 ('000 11 11 10', 1),
 ('001 01 01 00', 1),
 ('001 01 01 01', 1),
 ('001 10 10 00', 1),
 ('001 11 10 10', 1),
 ('010 00 11 00', 1),
 ('010 01 00 11', 1),
 ('010 01 10 01', 1),
 ('010 10 00 00', 1),
 ('010 10 01 00', 1),
 ('010 11 00 00', 1),
 ('010 11 01 00', 1),
 ('100 00 10 01', 1),
 ('100 10 10 00', 1)]

In [13]:
# Extract the mean and standard deviation out of the scaler_dict

scaler = scaler_dict[0]

mean = scaler.mean_
std = scaler.scale_

mean
std

array([79778555.89041053, 28557074.80284494])

## New get counts

In [14]:
# RESCALE IQ DATA

import warnings

from soft_info import get_repcode_IQ_map

def rescale_IQ_data(IQ_data, qubit_mapping, scaler_dict):
    scaled_IQ_data = []

    for shot in IQ_data:
        scaled_shot = []
        for idx, IQ_point in enumerate(shot):
            if qubit_mapping is not None and idx in qubit_mapping:
                qubit_idx = qubit_mapping[idx]
                scaler = scaler_dict.get(qubit_idx)

                if scaler is not None:
                    # Scale the IQ point
                    scaled_point = scaler.transform([[np.real(IQ_point), np.imag(IQ_point)]])
                    scaled_shot.extend(scaled_point[0])  # Flatten the 2D array to 1D and extend the shot -> [real1, imag1, real2, imag2, ..., realN, imagN]
                else:
                    # Handle the case where no scaler is found for the qubit_idx
                    warnings.warn(f"No scaler found for qubit {qubit_idx}. IQ data will be appended unscaled.")
                    scaled_shot.extend([np.real(IQ_point), np.imag(IQ_point)])
            else:
                # Handle the case where idx is not in qubit_mapping
                scaled_shot.extend([np.real(IQ_point), np.imag(IQ_point)])

        scaled_IQ_data.append(scaled_shot)

    return np.array(scaled_IQ_data)

# synd_rounds = 35
synd_rounds = 3
print("generating the qubit mapping")
qubit_mapping = get_repcode_IQ_map(layout, synd_rounds) #Hardcoded for repetition codes
print("rescaling the IQ data")
scaled_IQ_data = rescale_IQ_data(memory, qubit_mapping, scaler_dict)
print(scaled_IQ_data.shape)

generating the qubit mapping
rescaling the IQ data
(1000, 18)


In [15]:
counts = cpp_probabilities.get_counts_old(scaled_IQ_data, qubit_mapping, grid_dict, synd_rounds )
(sorted(counts.items(), key=lambda x: x[1], reverse=True))

[('000 00 00 00', 857),
 ('000 01 00 00', 18),
 ('000 00 01 00', 11),
 ('000 01 01 01', 11),
 ('000 10 10 10', 11),
 ('001 00 00 00', 8),
 ('100 00 10 00', 8),
 ('100 10 00 10', 7),
 ('000 10 00 00', 6),
 ('000 10 10 00', 6),
 ('001 01 00 01', 5),
 ('010 11 00 11', 5),
 ('000 00 00 01', 4),
 ('001 00 01 00', 4),
 ('001 01 00 00', 4),
 ('100 10 00 00', 4),
 ('000 00 10 00', 3),
 ('010 00 00 00', 3),
 ('100 00 00 00', 3),
 ('000 00 10 10', 2),
 ('000 01 01 00', 2),
 ('000 10 00 10', 2),
 ('000 00 00 10', 1),
 ('000 11 01 01', 1),
 ('000 11 11 10', 1),
 ('001 01 01 00', 1),
 ('001 01 01 01', 1),
 ('001 10 10 00', 1),
 ('001 11 10 10', 1),
 ('010 00 11 00', 1),
 ('010 01 00 11', 1),
 ('010 01 10 01', 1),
 ('010 10 00 00', 1),
 ('010 10 01 00', 1),
 ('010 11 00 00', 1),
 ('010 11 01 00', 1),
 ('100 00 10 01', 1),
 ('100 10 10 00', 1)]

# Basic testing

In [26]:
import random

# Assuming you have some function to create your grid data
def create_your_grid_data(num_points):
    # Replace this with your actual grid data creation logic
    grid_x, grid_y = np.meshgrid(np.linspace(0, 1, num_points), np.linspace(0, 1, num_points))
    grid_density_0 = np.random.rand(num_points, num_points)  # Placeholder values
    grid_density_1 = np.random.rand(num_points, num_points)  # Placeholder values
    return grid_x, grid_y, grid_density_0, grid_density_1

def create_specific_grid_data(num_points):
    # Create a meshgrid
    grid_x, grid_y = np.meshgrid(np.linspace(0, 1, num_points), np.linspace(0, 1, num_points))

    # Create grid densities: all zeros for grid_density_0 and all 0.5s for grid_density_1
    grid_density_0 = np.zeros((num_points, num_points))
    grid_density_1 = np.full((num_points, num_points), 0.5)

    return grid_x, grid_y, grid_density_0, grid_density_1


# Create instances of GridData for each qubit
kde_grid_dict = {}
for qubit_idx in range(2):  # Assuming two qubits
    grid_x, grid_y, grid_density_0, grid_density_1 = create_specific_grid_data(100)

    kde_grid_dict[qubit_idx] = cpp_probabilities.GridData(grid_x, grid_y, grid_density_0, grid_density_1)


def generate_random_qubit_mapping(num_keys):
    """
    Generates a random qubit mapping.

    Args:
    num_keys (int): The number of keys in the mapping.

    Returns:
    dict: A dictionary where keys are mapped randomly to either 0 or 1.
    """
    qubit_mapping = {}
    for i in range(num_keys):
        qubit_mapping[i] = random.randint(0, 1)
    return qubit_mapping

# Example usage
len_IQ = 9   # Number of keys in the mapping
qubit_mapping = generate_random_qubit_mapping(len_IQ)
print(qubit_mapping)


{0: 1, 1: 1, 2: 0, 3: 1, 4: 1, 5: 0, 6: 1, 7: 0, 8: 0}


In [27]:
# Example NumPy array (replace this with your actual data)
scaled_IQ_data_np = np.random.rand(int(1e2), len_IQ*2)  # 10 shots, len_IQ columns (2 measurements per shot)

# Convert NumPy array to Eigen::MatrixXd
scaled_IQ_data = cpp_probabilities.numpy_to_eigen(scaled_IQ_data_np)


# Example number of syndrome rounds
synd_rounds = 3

# Call the get_counts function
counts = cpp_probabilities.get_counts(scaled_IQ_data, qubit_mapping, kde_grid_dict, synd_rounds)
print(sorted(counts.items(), key=lambda x: x[1], reverse=True))


[('111 11 11 11', 100)]
